# ORGANICS

In [5]:
import numpy as np
import pandas as pd
import pylab as pl
import matplotlib.pyplot as plt
plt.style.use('ggplot')
%matplotlib inline
import seaborn as sns


In [21]:
data = pd.read_csv('ORGANICS.csv',encoding="cp1251")

In [22]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22223 entries, 0 to 22222
Data columns (total 13 columns):
ID                 22223 non-null int64
DemAffl            21138 non-null float64
DemAge             20715 non-null float64
DemCluster         21549 non-null float64
DemClusterGroup    21549 non-null object
DemGender          19711 non-null object
DemReg             21758 non-null object
DemTVReg           21758 non-null object
PromClass          22223 non-null object
PromSpend          22223 non-null float64
PromTime           21942 non-null float64
TargetBuy          22223 non-null int64
TargetAmt          22223 non-null int64
dtypes: float64(5), int64(3), object(5)
memory usage: 2.2+ MB


ID и TargetAmt нам не нужны

In [23]:
new_data = data[['DemAffl', 'DemAge', 'DemCluster', 'DemClusterGroup', 'DemGender', 'DemReg', 'DemTVReg', 'PromClass', 'PromSpend', 'PromTime', 'TargetBuy']]

In [25]:
new_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22223 entries, 0 to 22222
Data columns (total 11 columns):
DemAffl            21138 non-null float64
DemAge             20715 non-null float64
DemCluster         21549 non-null float64
DemClusterGroup    21549 non-null object
DemGender          19711 non-null object
DemReg             21758 non-null object
DemTVReg           21758 non-null object
PromClass          22223 non-null object
PromSpend          22223 non-null float64
PromTime           21942 non-null float64
TargetBuy          22223 non-null int64
dtypes: float64(5), int64(1), object(5)
memory usage: 1.9+ MB


In [26]:
new_data.shape

(22223, 11)

In [27]:
new_data.describe()

,DemAffl,DemAge,DemCluster,PromSpend,PromTime,TargetBuy
count,21138.000000,20715.000000,21549.000000,22223.000000,21942.000000,22223.000000
mean,8.711893,53.797152,27.193652,4420.590041,6.564670,0.247716
std,3.421125,13.206048,15.751547,7559.047522,4.657113,0.431696
min,0.000000,18.000000,1.000000,0.010000,0.000000,0.000000
25%,6.000000,44.000000,14.000000,0.010000,4.000000,0.000000
50%,8.000000,54.000000,27.000000,2000.000000,5.000000,0.000000
75%,11.000000,64.000000,38.000000,6000.000000,8.000000,0.000000
max,34.000000,79.000000,55.000000,296313.850000,39.000000,1.000000


In [28]:
data_corr = new_data.corr()

In [29]:
data_corr

,DemAffl,DemAge,DemCluster,PromSpend,PromTime,TargetBuy
DemAffl,1.000000,-0.137767,0.028571,-0.019621,-0.029736,0.359825
DemAge,-0.137767,1.000000,-0.172288,0.268434,0.207221,-0.302540
DemCluster,0.028571,-0.172288,1.000000,-0.108007,-0.054032,0.063521
PromSpend,-0.019621,0.268434,-0.108007,1.000000,0.033240,-0.079920
PromTime,-0.029736,0.207221,-0.054032,0.033240,1.000000,-0.053703
TargetBuy,0.359825,-0.302540,0.063521,-0.079920,-0.053703,1.000000


Проверка на пустые значения

In [30]:
new_data.isnull().any().any()

True

В некоторых строках отсутствуют данные.  Провести дальнейший анализ по двум ситуациям:
    1. Удаление Nan (dropna())
    2. Замена Nan усредненными значениями (fillna())

## Удаление Nan (dropna())

In [34]:
data1 = new_data.dropna()

In [35]:
data1.isnull().any().any()

False

In [45]:
x = data1[['DemAffl',  'DemAge', 'DemCluster', 'PromSpend', 'PromTime']]

In [46]:
y = data1.values[::,10]

In [47]:
print(x)

       DemAffl  DemAge  DemCluster  PromSpend  PromTime
0         10.0    76.0        16.0   16000.00       4.0
1          4.0    49.0        35.0    6000.00       5.0
2          5.0    70.0        27.0       0.02       8.0
3         10.0    65.0        51.0       0.01       7.0
4         11.0    68.0         4.0       0.01       8.0
...        ...     ...         ...        ...       ...
22216     13.0    49.0         8.0     500.00       9.0
22218     13.0    65.0         9.0    1500.00       5.0
22219     15.0    73.0        34.0    6053.06      12.0
22220      9.0    70.0        15.0    6000.00       5.0
22221     11.0    66.0         8.0    5000.00       5.0

[16408 rows x 5 columns]


In [48]:
print(y)

[0 0 1 ... 0 0 0]


Обучаем дерево:

In [37]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(x, y, test_size = 0.3)

y_train = y_train.astype(int)
y_test = y_test.astype(int)

In [38]:
from sklearn.tree import DecisionTreeClassifier

tree = DecisionTreeClassifier(max_depth = 5, criterion = 'entropy')
tree.fit(X_train, y_train)

DecisionTreeClassifier(class_weight=None, criterion='entropy', max_depth=5,
                       max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort=False,
                       random_state=None, splitter='best')

Визуализируем модель:

In [40]:
from sklearn.tree import export_graphviz

import subprocess

export_graphviz(tree, out_file='Organics_tree.dot', 
               rounded = True, proportion = False, 
                precision = 2, filled = True)

import pydot 
(graph, ) = pydot.graph_from_dot_file('Organics_tree.dot')

graph.write_png('Organics_tree.png')

FileNotFoundError: [WinError 2] "dot" not found in path.

Не работает, хотя всё верно, я уже не знаю, как решать эту проблему

In [49]:
from sklearn.metrics import classification_report, accuracy_score

clf = tree.fit(X_train, y_train)
y_pred = clf.predict(X_test)
result1 = classification_report(y_test, y_pred)

In [50]:
print("Классификация")
print (result1)

Классификация
              precision    recall  f1-score   support

           0       0.82      0.93      0.87      3644
           1       0.69      0.42      0.52      1279

    accuracy                           0.80      4923
   macro avg       0.75      0.68      0.70      4923
weighted avg       0.79      0.80      0.78      4923



In [51]:
result2 = accuracy_score(y_test,y_pred)
print("Accuracy:",result2)

Accuracy: 0.7997156205565712


Проверяем точность:

На обучающей

In [43]:
tree.score(X_train, y_train) 

0.8010448410970832

In [ ]:
На тестовой

In [52]:
tree.score(X_test,y_test)

0.7997156205565712

Посмотрим как параметры **max_depth** и **min_samples_leaf** влияют на построение дерева:

При **max_depth = 15**, **min_samples_leaf = 1**

In [54]:
tree_1 = DecisionTreeClassifier(max_depth = 15, min_samples_leaf = 1, criterion = 'entropy')
tree_1.fit(X_train, y_train)

DecisionTreeClassifier(class_weight=None, criterion='entropy', max_depth=15,
                       max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort=False,
                       random_state=None, splitter='best')

In [55]:
tree_1.score(X_train, y_train) 

0.8906399651719634

In [56]:
tree_1.score(X_test, y_test) 

0.7560430631728621

При **max_depth = 15**, **min_samples_leaf = 10**

In [58]:
tree_1_1 = DecisionTreeClassifier(max_depth = 15, min_samples_leaf = 10, criterion = 'entropy')
tree_1_1.fit(X_train, y_train)

DecisionTreeClassifier(class_weight=None, criterion='entropy', max_depth=15,
                       max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=10, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort=False,
                       random_state=None, splitter='best')

In [59]:
tree_1_1.score(X_train, y_train) 

0.8325642141924249

In [60]:
tree_1_1.score(X_test, y_test) 

0.7731058297785903

При **max_depth = 25**, **min_samples_leaf = 10**

In [61]:
tree_2 = DecisionTreeClassifier(max_depth = 25, min_samples_leaf = 10, criterion = 'entropy')
tree_2.fit(X_train, y_train)

DecisionTreeClassifier(class_weight=None, criterion='entropy', max_depth=25,
                       max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=10, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort=False,
                       random_state=None, splitter='best')

In [62]:
tree_2.score(X_train, y_train) 

0.8330866347409664

In [63]:
tree_2.score(X_test, y_test) 

0.7698557789965468

Влияние max_depth на дерево:

In [64]:
max_depth_values = range(1, 11)
scores_data = pd.DataFrame()

In [65]:
for max_depth in max_depth_values:    
    tree = DecisionTreeClassifier(criterion='entropy', max_depth=max_depth)
    tree.fit(X_train, y_train)
    train_score = tree.score(X_train, y_train)
    test_score = tree.score(X_test,y_test)
    
    temp_score_data = pd.DataFrame({'max_depth': [max_depth], 
                                    'train_score': [train_score],
                                    'test_score': test_score})
    scores_data = scores_data.append(temp_score_data)

In [66]:
scores_data

,max_depth,train_score,test_score
0,1,0.756639,0.766809
0,2,0.782586,0.790981
0,3,0.789726,0.795653
0,4,0.799216,0.803778
0,5,0.801045,0.799716
0,6,0.804005,0.801950
0,7,0.806966,0.799919
0,8,0.809752,0.797481
0,9,0.817849,0.792403
0,10,0.825424,0.788340


# Замена Nan усредненными значениями (fillna())

In [67]:
data2 = new_data.fillna(new_data.median(axis=0), axis=0)

In [68]:
x = data2[['DemAffl',  'DemAge', 'DemCluster', 'PromSpend', 'PromTime']]
y = data2.values[::,10]

In [69]:
print(x)

       DemAffl  DemAge  DemCluster  PromSpend  PromTime
0         10.0    76.0        16.0   16000.00       4.0
1          4.0    49.0        35.0    6000.00       5.0
2          5.0    70.0        27.0       0.02       8.0
3         10.0    65.0        51.0       0.01       7.0
4         11.0    68.0         4.0       0.01       8.0
...        ...     ...         ...        ...       ...
22218     13.0    65.0         9.0    1500.00       5.0
22219     15.0    73.0        34.0    6053.06      12.0
22220      9.0    70.0        15.0    6000.00       5.0
22221     11.0    66.0         8.0    5000.00       5.0
22222      3.0    75.0        53.0    6000.00       2.0

[22223 rows x 5 columns]


In [70]:
print(y)

[0 0 1 ... 0 0 0]


In [71]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(x, y, test_size = 0.3)

In [72]:
y_train = y_train.astype(int)
y_test = y_test.astype(int)

In [73]:
from sklearn.tree import DecisionTreeClassifier

tree2 = DecisionTreeClassifier(max_depth = 5, criterion = 'entropy')
tree2.fit(X_train, y_train)

DecisionTreeClassifier(class_weight=None, criterion='entropy', max_depth=5,
                       max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort=False,
                       random_state=None, splitter='best')

In [74]:
tree2.score(X_train, y_train) 

0.8024556441244536

In [75]:
tree2.score(X_test, y_test) 

0.8036598170091496

In [76]:
clf = tree2.fit(X_train, y_train)
y_pred2 = clf.predict(X_test)
result1 = classification_report(y_test, y_pred2)
result2 = accuracy_score(y_test,y_pred2)

In [77]:
print("Результат классификации:")
print (result1)

Результат классификации:
              precision    recall  f1-score   support

           0       0.83      0.92      0.88      4997
           1       0.66      0.44      0.53      1670

    accuracy                           0.80      6667
   macro avg       0.75      0.68      0.70      6667
weighted avg       0.79      0.80      0.79      6667



In [78]:
print("Accuracy:")
print(result2)

Accuracy:
0.8036598170091496


# Вопросы

**1.** Что такое решающее дерево? Как по построенному дереву найти прогноз для объекта?

Дерево решений - это некий бинарный классификатор

Правила классификации дерева дают прогноз каждому объекту дерева

**2.** Зачем в вершинах нужны предикаты? Какие типы предикатов вы знаете? 

Предикаты - это правила дерева которые делят информацию, классифицируют объект

Типы: Одномерные, многомерные

**3.** Почему для любой выборки можно построить решающее дерево, имеющее нулевую ошибку на ней?

**4.** Почему не рекомендуется строить небинарные деревья (т.е. имеющие больше двух потомков у каждой вершины)?

Оценка объектов по одному предикату не дает полного анализа данных

**5.** Как устроен жадный алгоритм построения дерева? Какие у него параметры?

Алгоритм каждый раз делает наилучший выбор и считает, что окончательное решение будет наилучшим. 

**6.** Зачем нужны критерии информативности?

Это функционал качества

Оценивает качество распределения